## Coupon Purchase Prediction

#### Ponpare (폰파레)

- 일본의 주요 소셜 커머스 사이트

this competition asks you to predict which coupons a customer will buy in a given period of time.

### Coupon_Area

In [1]:
coupon_area = pd.read_csv('coupon_area_train.csv')
coupon_area.tail()

,SMALL_AREA_NAME,PREF_NAME,COUPON_ID_hash
138180,新潟,新潟県,fe334f1a5e0dc54eca76b9dd9559661b
138181,新宿・高田馬場・中野・吉祥寺,東京都,8dc11b257cb1c5b98e7b7c66d9b2b779
138182,渋谷・青山・自由が丘,東京都,b15e2d7619c3fd9c6be567c971bdf4f4
138183,銀座・新橋・東京・上野,東京都,e1141a4b2798450f36e40090659c0b07
138184,新宿・高田馬場・中野・吉祥寺,東京都,79303de03e3af7105cded55960d62ac3


In [2]:
coupon_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138185 entries, 0 to 138184
Data columns (total 3 columns):
SMALL_AREA_NAME    138185 non-null object
PREF_NAME          138185 non-null object
COUPON_ID_hash     138185 non-null object
dtypes: object(3)
memory usage: 3.2+ MB


In [3]:
coupon_area.describe()

,SMALL_AREA_NAME,PREF_NAME,COUPON_ID_hash
count,138185,138185,138185
unique,55,47,19368
top,キタ,東京都,b4fd04b1d5585e2d5dc361e249a4812a
freq,3596,19720,55


### Coupon_Detail

In [4]:
coupon_detail = pd.read_csv('coupon_detail_train.csv')
coupon_detail.tail()

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash
168991,1,2012-02-07 12:14:50,池袋・神楽坂・赤羽,84b0c66349ae3c807f1d4601bfc0e8f6,280f0cedda5c4b171ee6245889659571,6eac7a7e347c563e5e00086f5eb47903
168992,1,2012-02-28 15:43:21,銀座・新橋・東京・上野,f7b2b854457ae6ece44be04c32520064,280f0cedda5c4b171ee6245889659571,a406e389e35e1140e4bc9b472d8258df
168993,1,2012-03-19 12:11:16,銀座・新橋・東京・上野,e12f28eb208f5466dede7a7cb2fc566b,280f0cedda5c4b171ee6245889659571,36addcc4b958135895c859d8783e3cd2
168994,2,2012-04-12 12:27:34,新宿・高田馬場・中野・吉祥寺,bcade77b186543a4820b3a6e3c06ad2f,280f0cedda5c4b171ee6245889659571,cb0244705306aafccc47bfe62ece39d3
168995,1,2012-05-09 12:12:26,新宿・高田馬場・中野・吉祥寺,fec51967a2f8135aa929cf2b5cc8722c,280f0cedda5c4b171ee6245889659571,31a605db6db5ad3fa3b2d4cf69ae3272


In [5]:
coupon_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168996 entries, 0 to 168995
Data columns (total 6 columns):
ITEM_COUNT         168996 non-null int64
I_DATE             168996 non-null object
SMALL_AREA_NAME    168996 non-null object
PURCHASEID_hash    168996 non-null object
USER_ID_hash       168996 non-null object
COUPON_ID_hash     168996 non-null object
dtypes: int64(1), object(5)
memory usage: 7.7+ MB


In [6]:
coupon_detail.describe()

,ITEM_COUNT
count,168996.000000
mean,1.430939
std,0.982732
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,55.000000


### Coupon_List

In [7]:
coupon_list = pd.read_csv('coupon_list_train.csv')
coupon_list.tail()

,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPFROM,DISPEND,DISPPERIOD,VALIDFROM,VALIDEND,...,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash
19408,ネイル・アイ,ネイル・アイ,65,10000,3500,2011-07-14 12:00:00,2011-07-17 12:00:00,3,2011-07-18,2011-12-30,...,1.0,1.0,1.0,1.0,1.0,1.0,北信越,新潟県,新潟,fe334f1a5e0dc54eca76b9dd9559661b
19409,グルメ,グルメ,50,5980,2990,2011-09-30 12:00:00,2011-10-03 12:00:00,3,2011-10-04,2012-02-29,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,東京都,新宿・高田馬場・中野・吉祥寺,8dc11b257cb1c5b98e7b7c66d9b2b779
19410,グルメ,グルメ,50,6300,3150,2011-09-03 12:00:00,2011-09-05 12:00:00,2,2011-09-06,2011-11-30,...,1.0,2.0,2.0,1.0,1.0,2.0,関東,東京都,渋谷・青山・自由が丘,b15e2d7619c3fd9c6be567c971bdf4f4
19411,グルメ,グルメ,50,2400,1200,2011-07-15 12:00:00,2011-07-17 12:00:00,2,2011-07-18,2011-10-31,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,東京都,銀座・新橋・東京・上野,e1141a4b2798450f36e40090659c0b07
19412,グルメ,グルメ,60,6000,2400,2011-07-23 12:00:00,2011-07-26 12:00:00,3,2011-07-27,2011-11-30,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,東京都,新宿・高田馬場・中野・吉祥寺,79303de03e3af7105cded55960d62ac3


In [8]:
coupon_list.describe()

,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPPERIOD,VALIDPERIOD,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY
count,19413.000000,19413.000000,19413.000000,19413.000000,13266.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000
mean,58.478391,11818.368258,4332.877659,3.166950,125.955902,0.936009,0.953901,0.982189,0.981867,1.000242,1.067618,0.921422,0.938024,1.044246
std,11.266571,16881.898880,5459.667448,1.346859,46.599249,0.309854,0.314045,0.217525,0.198105,0.319562,0.563260,0.424181,0.399619,0.534511
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,3675.000000,1550.000000,2.000000,89.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,53.000000,6500.000000,2750.000000,3.000000,128.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,65.000000,13650.000000,4800.000000,4.000000,177.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,100.000000,680000.000000,100000.000000,36.000000,179.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [9]:
coupon_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19413 entries, 0 to 19412
Data columns (total 24 columns):
CAPSULE_TEXT                  19413 non-null object
GENRE_NAME                    19413 non-null object
PRICE_RATE                    19413 non-null int64
CATALOG_PRICE                 19413 non-null int64
DISCOUNT_PRICE                19413 non-null int64
DISPFROM                      19413 non-null object
DISPEND                       19413 non-null object
DISPPERIOD                    19413 non-null int64
VALIDFROM                     13266 non-null object
VALIDEND                      13266 non-null object
VALIDPERIOD                   13266 non-null float64
USABLE_DATE_MON               12408 non-null float64
USABLE_DATE_TUE               12408 non-null float64
USABLE_DATE_WED               12408 non-null float64
USABLE_DATE_THU               12408 non-null float64
USABLE_DATE_FRI               12408 non-null float64
USABLE_DATE_SAT               12408 non-null float64
USAB

### Coupon_Visit

In [10]:
coupon_visit = pd.read_csv('coupon_visit_train.csv')
coupon_visit.tail()

,PURCHASE_FLG,I_DATE,PAGE_SERIAL,REFERRER_hash,VIEW_COUPON_ID_hash,USER_ID_hash,SESSION_ID_hash,PURCHASEID_hash
2833175,0,2011-08-20 12:13:21,4,ae75a8764951d2454273df8348dac8f3,8d91be4a8b5b02cf7117cb2b8e2d5a16,280f0cedda5c4b171ee6245889659571,f3261882332ac0d622d74ba067deb026,NaN
2833176,0,2011-08-20 12:19:05,7,48243ea50a35bb107fafd46a461b0b28,0fc7873f2a0bd451b97e97bd6b792ba9,280f0cedda5c4b171ee6245889659571,f3261882332ac0d622d74ba067deb026,NaN
2833177,0,2011-07-12 08:48:09,5,7d3892e54acb559ae36c459978489330,5cf5b9e5d600314a40a9fbac846ac15e,280f0cedda5c4b171ee6245889659571,cb624517ccc8fd90120bcfeb73d83758,NaN
2833178,0,2011-07-12 08:50:03,8,7d3892e54acb559ae36c459978489330,5cf5b9e5d600314a40a9fbac846ac15e,280f0cedda5c4b171ee6245889659571,cb624517ccc8fd90120bcfeb73d83758,NaN
2833179,0,2011-07-18 07:03:37,4,7d3892e54acb559ae36c459978489330,4ef82e25c2740e4ab55affbe5a003f50,280f0cedda5c4b171ee6245889659571,2a34fb9a7ea95bc5483d43a7295e5ee0,NaN


In [11]:
coupon_visit.describe()

,PURCHASE_FLG,PAGE_SERIAL
count,2.833180e+06,2.833180e+06
mean,4.319846e-02,7.654980e+00
std,2.033036e-01,1.215241e+01
min,0.000000e+00,1.000000e+00
25%,0.000000e+00,2.000000e+00
50%,0.000000e+00,4.000000e+00
75%,0.000000e+00,9.000000e+00
max,1.000000e+00,1.368000e+03


In [12]:
coupon_visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2833180 entries, 0 to 2833179
Data columns (total 8 columns):
PURCHASE_FLG           int64
I_DATE                 object
PAGE_SERIAL            int64
REFERRER_hash          object
VIEW_COUPON_ID_hash    object
USER_ID_hash           object
SESSION_ID_hash        object
PURCHASEID_hash        object
dtypes: int64(2), object(6)
memory usage: 172.9+ MB


### User_list

In [11]:
user_list = pd.read_csv('user_list.csv')
user_list.tail()

,REG_DATE,SEX_ID,AGE,WITHDRAW_DATE,PREF_NAME,USER_ID_hash
22868,2011-12-12 15:42:56,f,24,NaN,NaN,2f0a2f36a9f63b6ba2fa3a7e53bef906
22869,2011-08-10 00:49:55,m,41,NaN,京都府,6ae7811a9c7c58546d6a1567ab098c21
22870,2012-04-05 12:24:23,f,35,NaN,NaN,a417308c6a79ae0d86976401ec2e3b04
22871,2011-02-20 10:34:22,f,59,NaN,NaN,4937ec1c86e71d901c4ccc0357cff0b1
22872,2011-02-24 15:43:18,f,38,NaN,埼玉県,280f0cedda5c4b171ee6245889659571


In [12]:
user_list.describe()

,AGE
count,22873.000000
mean,42.497836
std,11.843465
min,15.000000
25%,33.000000
50%,41.000000
75%,51.000000
max,80.000000


In [13]:
user_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22873 entries, 0 to 22872
Data columns (total 6 columns):
REG_DATE         22873 non-null object
SEX_ID           22873 non-null object
AGE              22873 non-null int64
WITHDRAW_DATE    922 non-null object
PREF_NAME        15617 non-null object
USER_ID_hash     22873 non-null object
dtypes: int64(1), object(5)
memory usage: 1.0+ MB
